In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
train_df = pd.read_csv("working_data/trial_promo_training.csv")
test_df = pd.read_csv("working_data/trial_promo_testing.csv")

In [3]:
train_df["age"] = preprocessing.scale(train_df["age"])
train_df["income"] = preprocessing.scale(train_df["income"])
train_df["avbal"] = preprocessing.scale(train_df["avbal"])
train_df["avtrans"] = preprocessing.scale(train_df["avtrans"])
train_df.head()

,age,children,income,avbal,avtrans,decision,sexNum,mstatusNum,occupationNum,educationNum,...,occupation_finance,occupation_government,occupation_legal,occupation_manuf,occupation_medicine,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary
0,0.694301,2,-0.713177,-0.068324,0.547857,None,0,1,6,0,...,0,0,0,1,0,0,1,0,0,0
1,-1.466174,0,-0.713177,-0.068324,0.547857,None,1,2,1,2,...,0,0,0,0,0,0,0,0,1,0
2,0.737650,0,-1.267439,-0.178102,-0.509414,None,0,1,0,2,...,0,0,0,0,0,0,0,0,1,0
3,-1.096306,0,-0.713177,-0.068324,0.547857,A,1,1,1,2,...,0,0,0,0,0,0,0,0,1,0
4,-0.981640,0,-0.273744,-0.155999,-0.589653,None,1,2,2,3,...,0,0,0,0,0,0,0,0,0,1


In [4]:
X_train = train_df.loc[:, train_df.columns != "decision"]
y_train = train_df["decision"].replace("None", 0).replace("A", 1).replace("B", 2)
X_train.head()

,age,children,income,avbal,avtrans,sexNum,mstatusNum,occupationNum,educationNum,sex_F,...,occupation_finance,occupation_government,occupation_legal,occupation_manuf,occupation_medicine,occupation_retired,education_postgrad,education_professional,education_secondary,education_tertiary
0,0.694301,2,-0.713177,-0.068324,0.547857,0,1,6,0,1,...,0,0,0,1,0,0,1,0,0,0
1,-1.466174,0,-0.713177,-0.068324,0.547857,1,2,1,2,0,...,0,0,0,0,0,0,0,0,1,0
2,0.737650,0,-1.267439,-0.178102,-0.509414,0,1,0,2,1,...,0,0,0,0,0,0,0,0,1,0
3,-1.096306,0,-0.713177,-0.068324,0.547857,1,1,1,2,0,...,0,0,0,0,0,0,0,0,1,0
4,-0.981640,0,-0.273744,-0.155999,-0.589653,1,2,2,3,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
y_train.head()

0    1
1    2
2    1
3    1
4    0
Name: decision, dtype: int64

In [7]:
test_df["age"] = preprocessing.scale(test_df["age"])
test_df["income"] = preprocessing.scale(test_df["income"])
test_df["avbal"] = preprocessing.scale(test_df["avbal"])
test_df["avtrans"] = preprocessing.scale(test_df["avtrans"])
test_df.head()

X_test = test_df.loc[:, test_df.columns != "decision"]
y_test = test_df["decision"].replace("None", 0).replace("A", 1).replace("B", 2)
X_test.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,manuf,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans
0,1,0,1,0,0,0,1.694579,0,0,0,...,0,0,1,1,0,0,0,-1.055675,-1.348957,-0.814692
1,0,1,0,0,1,0,-1.051218,0,0,0,...,0,0,0,1,0,0,0,-1.009078,-0.330144,-0.691148
2,1,0,0,0,0,1,0.771610,2,0,0,...,0,0,0,0,1,0,0,0.222033,0.308487,-0.877786
3,1,0,0,0,1,0,-1.007801,0,0,0,...,0,0,0,0,1,0,0,0.487620,-0.646084,0.815205
4,1,0,0,0,0,1,0.340238,0,0,0,...,1,0,0,0,0,0,1,1.711120,-0.340193,1.348488


### Neural network

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [9]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)


y_test_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1, 2]))
print(accuracy_score(y_test, y_test_pred))

[[108  69  30]
 [  0  33   0]
 [  3   0   7]]
0.592


In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=42, n_estimators=1000)
clf.fit(X_train, y_train)

y_test_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_test_pred, labels=[0, 1, 2]))
print(accuracy_score(y_test, y_test_pred))

[[77 73 57]
 [ 0 33  0]
 [ 0  0 10]]
0.48
